In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import os

# Get the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = os.path.join(PROJECT_ROOT, CONFIG_FILE_PATH),
        params_filepath = os.path.join(PROJECT_ROOT, PARAMS_FILE_PATH)):

        # Convert string paths to Path objects
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        
        # Create artifacts directory at project root level
        self.artifacts_dir = os.path.join(PROJECT_ROOT, self.config.artifacts_root)
        create_directories([self.artifacts_dir])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        # Use absolute paths based on project root
        root_dir = os.path.join(PROJECT_ROOT, config.root_dir)
        local_data_file = os.path.join(PROJECT_ROOT, config.local_data_file)
        unzip_dir = os.path.join(PROJECT_ROOT, config.unzip_dir)
        
        create_directories([root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(local_data_file),
            unzip_dir=Path(unzip_dir)
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-03 10:59:34,411]: INFO: common: yaml file: e:\chickenproject\End-to-End-chicken-prediction\config\config.yaml loaded successfully]
[2025-11-03 10:59:34,425]: INFO: common: yaml file: e:\chickenproject\End-to-End-chicken-prediction\params.yaml loaded successfully]
[2025-11-03 10:59:34,425]: INFO: common: yaml file: e:\chickenproject\End-to-End-chicken-prediction\params.yaml loaded successfully]
[2025-11-03 10:59:34,428]: INFO: common: created directory at: artifacts]
[2025-11-03 10:59:34,430]: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-03 10:59:34,428]: INFO: common: created directory at: artifacts]
[2025-11-03 10:59:34,430]: INFO: common: created directory at: artifacts/data_ingestion]


[2025-11-03 10:59:45,197]: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5B3D:1B51CF:798EFE:CDDCC7:69083DBE
Accept-Ranges: bytes
Date: Mon, 03 Nov 2025 05:29:35 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210055-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1762147775.089788,VS0,VE804
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: bfaa98197c4c8dde85bf5c101f5904bad498529d
Expires: Mon, 03 Nov 2025 05:34:35 GMT
Source-Age: 1

]
